In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
152,ABW,North America,Aruba,2020-08-17,1102.0,54.0,77.000,4.0,0.0,0.143,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
383,AFG,Asia,Afghanistan,2020-08-17,37596.0,45.0,77.429,1375.0,5.0,9.000,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
532,AGO,Africa,Angola,2020-08-17,1879.0,27.0,43.857,86.0,0.0,2.286,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-16,France,217722.0,0.0,30447.0,0.0,0.0,0.0
0,2020-08-16,China,89341.0,0.0,4710.0,0.0,0.0,0.0
0,2020-08-16,Italy,254045.0,0.0,35408.0,0.0,0.0,0.0
0,2020-08-16,Spain,347350.0,0.0,28634.0,0.0,0.0,0.0
0,2020-08-16,United States,5449795.0,0.0,170813.0,0.0,0.0,0.0
0,2020-08-16,World,21901970.0,0.0,773784.0,0.0,0.0,0.0
0,2020-08-16,United Kingdom,317944.0,0.0,41422.0,0.0,0.0,0.0
0,2020-08-16,Germany,224761.0,0.0,9243.0,0.0,0.0,0.0
0,2020-08-16,Iran,345002.0,0.0,19738.0,0.0,0.0,0.0
0,2020-08-16,Turkey,249192.0,0.0,5974.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-17,France,217722.0,218536.0,30447.0,30410.0,-814.0,37.0
0,2020-08-17,China,89341.0,89375.0,4710.0,4703.0,-34.0,7.0
0,2020-08-17,Italy,254045.0,253915.0,35408.0,35396.0,130.0,12.0
0,2020-08-16,Spain,347350.0,342813.0,28634.0,28617.0,4537.0,17.0
0,2020-08-17,United States,5449795.0,5403269.0,170813.0,170052.0,46526.0,761.0
0,2020-08-17,World,21901970.0,21689832.0,773784.0,770273.0,212138.0,3511.0
0,2020-08-17,United Kingdom,317944.0,318484.0,41422.0,41366.0,-540.0,56.0
0,2020-08-17,Germany,224761.0,224014.0,9243.0,9232.0,747.0,11.0
0,2020-08-17,Iran,345002.0,343203.0,19738.0,19639.0,1799.0,99.0
0,2020-08-17,Turkey,249192.0,249309.0,5974.0,5974.0,-117.0,0.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")